In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import tempfile
import keras
from pathlib import Path

# Placement logs is disabled (default)
tf.debugging.set_log_device_placement(False)

2025-08-21 22:08:54.911101: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-21 22:08:54.938005: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-21 22:08:55.493183: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
for device in tf.config.list_physical_devices("GPU"):
    print(f"Device: {device.name}, Type: {device.device_type}")

Device: /physical_device:GPU:0, Type: GPU


# Basic

In [ ]:
# rank-0 tensor
x0 = tf.constant(4, dtype=tf.float16)
print(f"x0: {x0}")

# rank-1 tensor
x1 = tf.constant(
    [1.0, 2.0, 3.0],
    dtype=tf.float16)
print(f"x1: {x1}")

# rank-2 tensor
x2 = tf.constant([
    [1., 2., 3.],
    [4., 5., 6.]], dtype=tf.float16)
print(f"x2: {x2}")

print(f"Convert to numpy array: {type(x2.numpy())}")

In [ ]:
# Convert some object to tensor
y = tf.convert_to_tensor([1,2,3])
print(f"Value: {y}")

In [ ]:
a = tf.constant([[1, 2],
                 [3, 4]])
b = tf.ones([2, 2], dtype=tf.int32)

print(f"Add: {tf.add(a, b)}") # or a + b
print(f"Mul: {tf.multiply(a, b)}") # or a * b
print(f"Matrix mul: {tf.matmul(a, b)}") # or a @ b

In [ ]:
c = tf.constant([[4.0, 5.0],
                 [10.0, 1.0]])
print(f"Largest value: {tf.reduce_max(c)}")
print(f"The index of largest value: {tf.math.argmax(c)}")
print(f"Softmax: {tf.nn.softmax(c)}")

In [ ]:
# Reshape
x = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], dtype=tf.float16)
print(f"Shape: {x.shape}")
y = tf.reshape(x, [3, 2])
print(f"ReShape: {y}")


In [ ]:
# Define mutable tensor (represents shared, persistent state to manipulate of)
# Usage: storing model parameters (e.g. weights)
z = tf.Variable([0.0, 0.0, 0.0])
print(f"Value: {z}")
z.assign([1.0, 2.0, 3.0])
print(f"Value: {z}")

In [ ]:
# Define not rectangular tensor
z = tf.ragged.constant([
    [0, 1, 2, 3],
    [4, 5],
    [6, 7, 8],
    [9]
])
print(f"Not-rectangular tensor: {z}")
print(f"Not-rectangular tensor (shape): {z.shape}")

In [ ]:
# Define sparce tensor
s = tf.sparse.SparseTensor(indices=[[0, 0], [1, 2]],
                           values=[1, 2],
                           dense_shape=[3, 4])
print(f"Sparse: {s}")
print(f"Sparce to dence: {tf.sparse.to_dense(s)}")

In [ ]:
# Logging of device location must be set from session start
with tf.device("CPU:0"):
  a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
  c = tf.matmul(a, b)
print(f"Result: {c}")

# Automatic differentiation

In [4]:
#
# AD with respect to scalar
#
x = tf.Variable(3.0)

with tf.GradientTape() as tape:
    y = x**2

dx = tape.gradient(y, x)
print(f"Result: {dx.numpy()}")

Result: 6.0


In [4]:
#
# AD with respect to tensor
#
w = tf.Variable(tf.random.normal((3, 2)), name='w')
b = tf.Variable(tf.zeros(2, dtype=tf.float32), name='b')
x = [[1., 2., 3.]]

with tf.GradientTape(persistent=True) as tape:
  y = x @ w + b
  loss = tf.reduce_mean(y**2)

grad = tape.gradient(loss, {
    "w": w,
    "b": b,
})
print(f"w.shape = {w.shape}")
print(f"dw.shape = {grad["w"].shape}")

w.shape = (3, 2)
dw.shape = (3, 2)


In [ ]:
#
# AD with respect to a model
#

layer = tf.keras.layers.Dense(2, activation='relu')
x = tf.constant([[1., 2., 3.]])

with tf.GradientTape() as tape:
    # Forward propagation
    y = layer(x)
    loss = tf.reduce_mean(y**2)

# Back propagation
grad = tape.gradient(loss, layer.trainable_variables)

for var, g in zip(layer.trainable_variables, grad):
    print(f"{var.name}, shape: {g.shape}")

In [ ]:
#
# AD with selecting what to watch to calculate gradients against
#

x0 = tf.Variable(0.0)
x1 = tf.Variable(10.0)

with tf.GradientTape(watch_accessed_variables=False) as tape:
  tape.watch(x1)
  y0 = tf.math.sin(x0)
  y1 = tf.nn.softplus(x1)
  y = y0 + y1
  ys = tf.reduce_sum(y)

# dys/dx1 = exp(x1) / (1 + exp(x1)) = sigmoid(x1)
grad = tape.gradient(ys, {'x0': x0, 'x1': x1})

print('dy/dx0:', grad['x0'])
print('dy/dx1:', grad['x1'].numpy())

In [ ]:
# Save and restore model (all variables declared inside)
class ExampleModel(tf.Module):
    def __init__(self, value):
        super().__init__()
        self.weights = tf.Variable(value)

    @tf.function
    def mul(self, x):
        return x * self.weights

module = ExampleModel(3)
tf.print(module.mul(tf.constant([1, 2, 3])))

save_path = tempfile.gettempdir()
tf.saved_model.save(module, save_path)

reloaded = tf.saved_model.load(save_path)
tf.print(reloaded.mul(tf.constant([1, 1, 1])))

# Graph

In [ ]:
def inner_fn(x, y, b):
    x = tf.matmul(x, y)
    x = x + b
    return x

# Decorator makes function into a `PolymorphicFunction`
# as well applying this transformation to other functions it calls
@tf.function
def outer_fn(x):
    y = tf.constant([[2.0], [3.0]])
    b = tf.constant(4.0)

    return inner_fn(x, y, b)


print(f"Type: {type(outer_fn)}")


result = outer_fn(tf.constant([[1.0, 2.0]]))
print(f"Result: {result}")

In [ ]:
def simple_relu(x):
    if tf.greater(x, 0):
        return x
    else:
        return 0

tf_simple_relu = tf.function(simple_relu)

print(">>> Graph-generated output of AutoGraph:")
print(tf.autograph.to_code(simple_relu))

print(">>> Graph:")
print(tf_simple_relu.get_concrete_function(tf.constant(1)).graph.as_graph_def())

In [ ]:
# Turnt off running graphs and force to run as ordinary function
tf.config.run_functions_eagerly(True)
simple_relu(tf.constant(-1.0))
tf.config.run_functions_eagerly(False)

In [ ]:
# Tracing captures the TensorFlow operations and discard all other
# We observe only one print statement when tf.function runs the original code
@tf.function
def get_MSE(y_true, y_pred):
  print("Calculating MSE!")
  sq_diff = tf.pow(y_true - y_pred, 2)
  return tf.reduce_mean(sq_diff)

y_true = tf.random.uniform([5], maxval=10, dtype=tf.int32)
y_pred = tf.random.uniform([5], maxval=10, dtype=tf.int32)

error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)

In [ ]:
import timeit

# Ordinary functions
def power(x, y):
  result = tf.eye(10, dtype=tf.dtypes.int32)
  for _ in range(y):
    result = tf.matmul(x, result)
  return result

# Grapth function
power_as_graph = tf.function(power)

x = tf.random.uniform(shape=[10, 10], minval=-1, maxval=2, dtype=tf.dtypes.int32)
print("Eager execution:", timeit.timeit(lambda: power(x, 100), number=1000), "seconds")

power_as_graph = tf.function(power)
print("Graph execution:", timeit.timeit(lambda: power_as_graph(x, 100), number=1000), "seconds")

# Module

## Using module

In [ ]:
class MyModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.var1 = tf.Variable(5.0, name="train_me")
        self.var2 = tf.Variable(5.0, trainable=False, name="do_not_train_me")

    @tf.function
    def __call__(self, x):
        return self.var1 * x + self.var2

In [ ]:
model1 = MyModule(name="simple")

model1(tf.constant(5.0))

In [ ]:
# All trainable variables
print("Trainable variables:", model1.trainable_variables)
# Every variable
print("All variables:", model1.variables)

In [ ]:
class Dense(tf.Module):
    def __init__(self, in_features, out_features, name=None):
        super().__init__(name=name)
        tf.random.set_seed(0)
        self.w = tf.Variable(tf.random.normal([in_features, out_features]), name='w')
        self.b = tf.Variable(tf.zeros([out_features]), name='b')

    @tf.function
    def __call__(self, x):
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)

class SequentialModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.dense_1 = Dense(in_features=3, out_features=3)
        self.dense_2 = Dense(in_features=3, out_features=2)

    @tf.function
    def __call__(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)

In [ ]:
# You have made a model!
model2 = SequentialModule(name="model2")
print("Model results (model2):", model2(tf.constant([[2.0, 2.0, 2.0]])))

In [ ]:
# Every module
print("All submodules:", model2.submodules)
# Every variable
print("All variables:", model2.variables)

## Saving checkpoint

In [ ]:
export_path = Path(tempfile.gettempdir()) / "checkpoint"
checkpoint1 = tf.train.Checkpoint(model=model2)
checkpoint1.write(export_path)

In [ ]:
tf.train.list_variables(export_path)

In [ ]:
model3 = SequentialModule(name="model3")
checkpoint2 = tf.train.Checkpoint(model=model3)
checkpoint2.restore(export_path)

model3(tf.constant([[2.0, 2.0, 2.0]]))
print("Model results (model3):", model3(tf.constant([[2.0, 2.0, 2.0]])))

## Saving model

In [ ]:
# Saved model contains both a collection of functions and a collection of weights
export_path = Path(tempfile.gettempdir()) / "saved_model"
tf.saved_model.save(model2, export_path)

In [ ]:
%ls -l {export_path}

In [ ]:
# The variable directory contains a checkpoint of the variables
%ls -l {export_path / "variables"}

In [ ]:
model4 = tf.saved_model.load(export_path)

# loaded model is an internal TF user object without any of class knowledge
print("Is instance of SequentialModule:", isinstance(model4, SequentialModule))

In [ ]:
print(model4(tf.constant([[2.0, 2.0, 2.0]])))

## Create layer using subclassing from Keras

In [ ]:
class FlexibleDense(tf.keras.layers.Layer):
    # Note the added `**kwargs`, as Keras supports many arguments
    def __init__(self, out_features, **kwargs):
        super().__init__(**kwargs)
        self.out_features = out_features

    # Create the state of the layer (weights)
    def build(self, input_shape):
        self.w = tf.Variable(tf.random.normal([input_shape[-1], self.out_features]), name='w')
        self.b = tf.Variable(tf.zeros([self.out_features]), name='b')

    # Defines the computation from inputs to outputs
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

@keras.saving.register_keras_serializable()
class FlexibleSequentialModel(tf.keras.Model):
  def __init__(self, name=None, **kwargs):
    super().__init__(**kwargs)
    self.dense_1 = FlexibleDense(out_features=3)
    self.dense_2 = FlexibleDense(out_features=2)

  def call(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

In [ ]:
# Create the instance of the layer
model5 = FlexibleSequentialModel(name="model5")
print("Model results:", model5(tf.constant([[2.0, 2.0, 2.0]])))

### Saving Keras model

In [ ]:
export_path = Path(tempfile.gettempdir()) / "model5.keras"
model5.save(export_path)

In [ ]:
model6 = keras.models.load_model(export_path)
print("Reconstructed model results:", model6(tf.constant([[2.0, 2.0, 2.0]])))

# Training loop

In [ ]:
TRUE_W = 3.0
TRUE_B = 2.0
NUM_EXAMPLES = 201

# A vector of random x values
x = tf.linspace(-2,2, NUM_EXAMPLES)
x = tf.cast(x, tf.float32)

def f(x):
  return x * TRUE_W + TRUE_B

# Generate some noise
noise = tf.random.normal(shape=[NUM_EXAMPLES])

# Calculate y
y = f(x) + noise

In [ ]:
# Plot all the data
plt.plot(x, y, '.', label="Data")
plt.plot(x, f(x), label="Ground truth")
plt.legend()
plt.show()

## Training without Keras

In [ ]:
class CustomModel(tf.Module):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)
    self.w = tf.Variable(5.0)
    self.b = tf.Variable(0.0)

  def __call__(self, x):
    return self.w * x + self.b

In [ ]:
# Define the model
model7 = CustomModel()

# List
print("Variables: {}".format(model7.variables))

# Verify the model works
assert model7(3.0).numpy() == 15.0

In [ ]:
# Define loss function
def loss(target_y, predicted_y):
  return tf.reduce_mean(tf.square(target_y - predicted_y))

# Define train function
def train(model, x, y, learning_rate):
    with tf.GradientTape() as t:
        # Trainable variables are automatically tracked by GradientTape
        current_loss = loss(y, model(x))

    # Use GradientTape.gradient to calculate the gradients with respect to w and b
    dw, db = t.gradient(current_loss, [model.w, model.b])

    # Subtract the gradient scaled by the learning rate
    model.w.assign_sub(learning_rate * dw)
    model.b.assign_sub(learning_rate * db)

In [ ]:
# Collect the history of W-values and b-values to plot later
weights = []
biases = []
epochs = range(10)

# Define a training loop
def report(model, loss):
  return f"W = {model.w.numpy():1.2f}, b = {model.b.numpy():1.2f}, loss={loss:2.5f}"

def training_loop(model, x, y):
  for epoch in epochs:
    # Update the model with the single giant batch
    train(model, x, y, learning_rate=0.1)

    # Track this before I update
    weights.append(model.w.numpy())
    biases.append(model.b.numpy())
    current_loss = loss(y, model(x))

    print(f"Epoch {epoch:2d}:")
    print("    ", report(model, current_loss))

In [ ]:
plt.plot(x, y, '.', label="Data")
plt.plot(x, f(x), label="Ground truth")
plt.plot(x, model7(x), label="Predictions")
plt.legend()
plt.show()

print("Current loss: %1.6f" % loss(y, model7(x)).numpy())

In [ ]:
current_loss = loss(y, model7(x))

print(f"Starting:")
print("    ", report(model7, current_loss))

training_loop(model7, x, y)

In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

plt.plot(epochs, weights, label='Weights', color=colors[0])
plt.plot(epochs, [TRUE_W] * len(epochs), '--',
         label = "True weight", color=colors[0])

plt.plot(epochs, biases, label='bias', color=colors[1])
plt.plot(epochs, [TRUE_B] * len(epochs), "--",
         label="True bias", color=colors[1])

plt.legend()
plt.show()

In [ ]:
plt.plot(x, y, '.', label="Data")
plt.plot(x, f(x), label="Ground truth")
plt.plot(x, model7(x), label="Predictions")
plt.legend()
plt.show()

print("Current loss: %1.6f" % loss(model7(x), y).numpy())

## Training with Keras

In [ ]:
class CustomKerasModel(keras.Model):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)
    self.w = tf.Variable(5.0)
    self.b = tf.Variable(0.0)

  def call(self, x):
    return self.w * x + self.b

In [ ]:
model8 = CustomKerasModel()

# compile sets the training parameters
model8.compile(
    # By default, fit() uses tf.function().  You can
    # turn that off for debugging, but it is on now.
    run_eagerly=False,

    # Using a built-in optimizer, configuring as an object
    optimizer=keras.optimizers.SGD(learning_rate=0.1),

    # Keras comes with built-in MSE error
    # However, you could use the loss function
    # defined above
    loss=keras.losses.mean_squared_error,
)

In [ ]:
print(x.shape[0])
model8.fit(x, y, epochs=10, batch_size=1000)